This example mirrors [mesh.curv.html](https://niivue.com/demos/features/mesh.curv.html).

In [ ]:
from pathlib import Path
from ipyniivue import download_dataset

# 1. Download

# Matcaps
BASE_API_URL = "https://niivue.com/demos/"
MATCAPS_DATA_FOLDER = Path("matcaps")
download_dataset(
    api_url=BASE_API_URL + "matcaps",
    dest_folder=MATCAPS_DATA_FOLDER,
    files=[
        "Shiny.jpg",
        "Cortex.jpg",
        "Cream.jpg",
        "Fuzzy.jpg",
        "Peach.jpg",
        "Plastic.jpg",
        "Gold.jpg",
    ],
)

# Images
IMAGES_DATA_FOLDER = Path("images")
download_dataset(
    api_url=BASE_API_URL + "images",
    dest_folder=IMAGES_DATA_FOLDER,
    files=[
        "fs_LR.32k.LR.curvature.dscalar.nii",
        "fs_LR.32k.L.inflated.surf.gii",
    ],
)

In [ ]:
import ipywidgets as widgets
import ipyniivue

# 2. Initialize NiiVue
nv = ipyniivue.NiiVue()
nv.opts.loading_text = "there are no images"
nv.opts.back_color = [1, 1, 1, 1]
nv.opts.show_3d_crosshair = True

# 3. UI Controls

# Slider for opacity (Curve)
opacity_slider = widgets.FloatSlider(
    min=0, max=1.0, value=222 / 255, description="Curve", continuous_update=True
)

# Checkbox for HighDPI
high_dpi_check = widgets.Checkbox(value=False, description="HighDPI")

# Dropdown for MatCaps
matcap_names = ["Shiny", "Cortex", "Cream", "Fuzzy", "Peach", "Plastic"]
matcap_dropdown = widgets.Dropdown(
    options=matcap_names, value="Shiny", description="MatCap"
)

# Dropdown for Shaders
shader_names = nv.mesh_shader_names()
shader_dropdown = widgets.Dropdown(
    options=shader_names, value="Matcap", description="Shaders"
)

# 4. Callbacks

def on_opacity_change(change):
    """Handle opacity change."""
    if nv.meshes:
        nv.set_mesh_layer_property(nv.meshes[0].id, 0, "opacity", change["new"])


def on_high_dpi_change(change):
    """Handle dpi change."""
    nv.set_high_resolution_capable(bool(change["new"]))


def on_matcap_change(change):
    """Handle matcap change."""
    if nv.meshes:
        nv.set_mesh_shader(nv.meshes[0].id, "Matcap")
        shader_dropdown.value = "Matcap"
        name = change["new"]
        matcap_path = MATCAPS_DATA_FOLDER / f"{name}.jpg"
        with open(matcap_path, "rb") as f:
            matcap_data = f.read()
        nv.load_mat_cap_texture(matcap_data)


def on_shader_change(change):
    """Handle shader change."""
    if nv.meshes:
        nv.set_mesh_shader(nv.meshes[0].id, change["new"])


# Attach handlers
opacity_slider.observe(on_opacity_change, names="value")
high_dpi_check.observe(on_high_dpi_change, names="value")
matcap_dropdown.observe(on_matcap_change, names="value")
shader_dropdown.observe(on_shader_change, names="value")

# 5. Data Loading
mesh_layer = {
    "path": IMAGES_DATA_FOLDER / "fs_LR.32k.LR.curvature.dscalar.nii",
    "colormap": "gray",
    "cal_min": -0.15,
    "cal_max": -0.001,
    "opacity": 222 / 255,
}

nv.load_meshes(
    [
        {
            "path": IMAGES_DATA_FOLDER / "fs_LR.32k.L.inflated.surf.gii",
            "layers": [mesh_layer],
        }
    ]
)

# 6. Initialize State
on_matcap_change({"new": "Shiny"})

# 6. Display All
header = widgets.HBox([opacity_slider, high_dpi_check])
footer = widgets.HBox([shader_dropdown, matcap_dropdown])
widgets.VBox([header, nv, footer])